<h1>Final Capstone Project: Hockey Clubs in Belgium. Where do they thrive best? </h1>

<p>Field Hockey is a booming sport in Belgium, with our national hockey team being nr 1 in the world.  New clubs are being opened.  

<br>How is the distribution of hockey clubs across Flanders?
<br>By plotting the Flanders communes and the hockeyclubs in their proximity, we can visualize where there is potential for starting a new club.
<br>Using foursquare data we can see if in areas with no hockey clubs, there is potential to get buy in from local city to establish a new club.

<br>The following data will be used
<br> Excel file downloaded from www including all hockey clubs in Flanders/Belgium with their address and number of members
<br> Wikipedia list of Flanders communes.   Using screenscraping to use in dataframe
<br> Foursquare API to list all sports venues in neighbourhood of larger cities with no hockey presence

 </p>

<h1>Notebook preparation</h1>

In [1]:
!conda install -c conda-forge geopy --yes 
!conda install -c conda-forge folium=0.5.0 --yes
!conda install lxml
!conda install requests

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.9.11  |       hecc5488_0         144 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    cer

In [2]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library


In [3]:
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [4]:
from bs4 import BeautifulSoup
import json # library to handle JSON files
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

<h1>Flanders communes</h1>

<h2>Display Flanders communes on map</h2>

In [5]:
#Extract communes from wikipedia,  use Beautifulsoup, and read to dataframe
website_URL=requests.get('https://nl.wikipedia.org/wiki/Lijst_van_gemeenten_in_het_Vlaams_Gewest').text
soup = BeautifulSoup(website_URL,'lxml')
My_table = soup.find('table',{'class':'wikitable sortable'}).prettify()
dfs = pd.read_html(My_table)
df=dfs[0]
df.head()

,Gemeente,Stad/ Gem.,Provincie,Opper- vlakte (km²),Aantal inwoners (01/01/2016),Deelgemeenten
0,Aalst,stad,Oost-Vlaanderen,7812,84.329,"Aalst, Gijzegem , Hofstade , Baardegem ,..."
1,Aalter [1],gem.,Oost-Vlaanderen,11919,28.435,"Aalter, Bellem , Knesselare , Lotenhulle ..."
2,Aarschot,stad,Vlaams-Brabant,6252,29.529,"Aarschot, Gelrode , Langdorp , Rillaar"
3,Aartselaar,gem.,Antwerpen,1093,14.262,Aartselaar
4,Affligem,gem.,Vlaams-Brabant,1770,13.079,"Essene , Hekelgem , Teralfene"


In [9]:
#Using geopy to add longitude and lattitude data to plot on map 
#Add coordinates and clean up
geolocator = Nominatim(user_agent="foursquare_agent")

from geopy.extra.rate_limiter import RateLimiter
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
df['location'] = df['Gemeente'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df['point'] = df['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df['point'].tolist(), index=df.index)

df_clean = df.drop(["point", "altitude", "location"], axis=1)
#Drop NA records and sort
df_clean = df_clean.dropna()
df_clean = df_clean.sort_values(by=df_clean.columns[4], ascending=False)
df_clean.shape

(293, 8)

In [10]:
df.head(20)

,Gemeente,Stad/ Gem.,Provincie,Opper- vlakte (km²),Aantal inwoners (01/01/2016),Deelgemeenten,location,point,latitude,longitude,altitude
0,Aalst,stad,Oost-Vlaanderen,7812,84.329,"Aalst, Gijzegem , Hofstade , Baardegem ,...","(Aalst, Oost-Vlaanderen, Vlaanderen, België - ...","(50.9383226, 4.0392147, 0.0)",50.938323,4.039215,0.0
1,Aalter [1],gem.,Oost-Vlaanderen,11919,28.435,"Aalter, Bellem , Knesselare , Lotenhulle ...",None,None,NaN,NaN,NaN
2,Aarschot,stad,Vlaams-Brabant,6252,29.529,"Aarschot, Gelrode , Langdorp , Rillaar","(Aarschot, Leuven, Vlaams-Brabant, Vlaanderen,...","(50.98550925, 4.83374800692363, 0.0)",50.985509,4.833748,0.0
3,Aartselaar,gem.,Antwerpen,1093,14.262,Aartselaar,"(Aartselaar, Antwerpen, Vlaanderen, 2630, Belg...","(51.133297, 4.3870241, 0.0)",51.133297,4.387024,0.0
4,Affligem,gem.,Vlaams-Brabant,1770,13.079,"Essene , Hekelgem , Teralfene","(Affligem, Halle-Vilvoorde, Vlaams-Brabant, Vl...","(50.9035063, 4.1174675, 0.0)",50.903506,4.117468,0.0
5,Alken,gem.,Limburg,2814,11.456,Alken,"(Alken, Tongeren, Limburg, Vlaanderen, 3570, B...","(50.8754731, 5.3079873, 0.0)",50.875473,5.307987,0.0
6,Alveringem,gem.,West-Vlaanderen,8001,5.039,"Alveringem, Hoogstade , Oeren , Sint-Rijk...","(Alveringem, Veurne, West-Vlaanderen, Vlaander...","(51.01509585, 2.72256881027274, 0.0)",51.015096,2.722569,0.0
7,Antwerpen,stad,Antwerpen,20451,517.042,"Antwerpen , Berendrecht-Zandvliet-Lillo , ...","(Antwerpen, Vlaanderen, België - Belgique - Be...","(51.2211097, 4.3997081, 0.0)",51.221110,4.399708,0.0
8,Anzegem,gem.,West-Vlaanderen,4179,14.589,"Anzegem, Gijzelbrechtegem , Ingooigem , V...","(Anzegem, Kortrijk, West-Vlaanderen, Vlaandere...","(50.8436655, 3.46422356526908, 0.0)",50.843666,3.464224,0.0
9,Ardooie,gem.,West-Vlaanderen,3458,9.053,"Ardooie, Koolskamp","(Ardooie, Tielt, West-Vlaanderen, Vlaanderen, ...","(50.9797876, 3.1974259, 0.0)",50.979788,3.197426,0.0


In [11]:
df_clean.head(20)

,Gemeente,Stad/ Gem.,Provincie,Opper- vlakte (km²),Aantal inwoners (01/01/2016),Deelgemeenten,latitude,longitude
7,Antwerpen,stad,Antwerpen,20451,517.042,"Antwerpen , Berendrecht-Zandvliet-Lillo , ...",51.221110,4.399708
75,Gent,stad,Oost-Vlaanderen,15618,257.029,"Gent, Mariakerke , Drongen , Wondelgem ,...",51.053829,3.725012
45,Brugge,stad,West-Vlaanderen,13840,118.053,"Brugge, Koolkerke , Sint-Andries , Sint-M...",51.208553,3.226772
156,Leuven,stad,Vlaams-Brabant,5663,99.288,"Leuven, Heverlee , Kessel-Lo , Wilsele ,...",50.879202,4.701168
102,Herstappe,gem. [2],Limburg,135,89.000,Herstappe,50.727383,5.426092
179,Mechelen,stad,Antwerpen,6519,84.523,"Mechelen, Walem , Heffen , Hombeek , Le...",51.028138,4.480345
0,Aalst,stad,Oost-Vlaanderen,7812,84.329,"Aalst, Gijzegem , Hofstade , Baardegem ,...",50.938323,4.039215
91,Hasselt,stad,Limburg,10224,76.685,"Hasselt, Sint-Lambrechts-Herk , Wimmertinge...",50.930374,5.337804
140,Kortrijk,stad,West-Vlaanderen,8002,75.506,"Kortrijk, Bissegem , Heule , Bellegem , ...",50.827643,3.265988
240,Sint-Niklaas,stad,Oost-Vlaanderen,8380,75.208,"Sint-Niklaas, Nieuwkerken-Waas , Belsele ,...",51.164668,4.139512


In [13]:
#map of Flanders with Brussels as center
address = 'Brussels, Belgium'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

# create map of Flanders with Brussels in center, using latitude and longitude values
map_flanders = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, gemeente in zip(df_clean['latitude'], df_clean['longitude'], df_clean['Gemeente']):
    label = '{}'.format(gemeente)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_flanders)  
    
map_flanders

<h1>Hockey clubs in Flanders</h1>

In [16]:
# The code was removed by Watson Studio for sharing.

,Club,Website,Adres,E-mail,Hockey,Leden
0,ARA Gantoise HC,www.gantoise.be,"Tennisstraat 13, 9050 Gent, België",hockey@gantoise.be,Recr. Comp.,1349.0
1,Blackbirds HC,www.hcblackbirds.be,"Voetbalstraat 11- 13, 2240 Zandhoven, België",secretarisblackbirds@gmail.com,Recr. Comp.,235.0
2,Blue Lions Tervuren HC,www.bluelions.be,"Pater Dupierreuxlaan 1, Tervuren 3080 Tervuren...",secretaris@bluelions.be,Recr. Comp.,439.0
3,Braxgata HC,www.braxgata.be,"Schommelei 1-4, 2850 Boom, België",secretaris@braxgata.be,Recr. Comp.,1290.0
4,Constantia HC,www.constantia.info,"Bergstraat 41,, België",elewin@waregemsportcenter.be,Recr. Comp.,338.0


In [17]:
#geopy for geo data for mapping
from geopy.extra.rate_limiter import RateLimiter
# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
# 2- - create location column
df_data_1['location'] = df_data_1['Adres'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df_data_1['point'] = df_data_1['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df_data_1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df_data_1['point'].tolist(), index=df_data_1.index)
#df_data_1.head()


In [20]:
#more clean up
# Delete multiple columns from the dataframe
df_data_clean = df_data_1.drop(["point", "altitude", "location"], axis=1)
#Sort on number of members "Leden"
df_data_clean = df_data_clean.sort_values(by="Leden", ascending=False)
#Drop NA records
df_data_clean = df_data_clean.dropna()
df_data_clean.head(10)

,Club,Website,Adres,E-mail,Hockey,Leden,latitude,longitude
46,R Victory HC,www.victory.be,"Kattenbroek 5, 2650 Edegem, België",info@victory.be,Recr. Comp.,1589.0,51.149110,4.430133
30,KHC Dragons,www.dragons.be,"Gemeentepark 12, 2930 Brasschaat, België",secretaris@dragons.be,Recr. Comp.,1426.0,51.284169,4.488786
0,ARA Gantoise HC,www.gantoise.be,"Tennisstraat 13, 9050 Gent, België",hockey@gantoise.be,Recr. Comp.,1349.0,51.030502,3.766894
3,Braxgata HC,www.braxgata.be,"Schommelei 1-4, 2850 Boom, België",secretaris@braxgata.be,Recr. Comp.,1290.0,51.089335,4.375491
26,Indiana THC,www.thc-indiana.be,"Hondelestraat 13,9840 De Pinte, België",hockey@thc-indiana.be,Recr. Comp.,1103.0,50.993195,3.693569
31,KHC Leuven,www.khcl.be/home3.asp?ClubID=83&LG=NL,"Hertogstraat 203,3001 Heverlee, België",secretariaat@khcl.be,Recr. Comp.,1084.0,50.856356,4.706686
44,R Herakles HC,www.herakles.be,"Hockeyweg 1, 2500 Lier, België",secretariaat@herakles.be,Recr. Comp.,984.0,51.144168,4.531690
28,K Mechelse THC,www.kmthc.be,"Bergenstraat 2,2811 Hombeek, België",annemay.meijburg@,Recr. Comp.,879.0,51.011183,4.442078
29,KHC Brugge,www.khcbrugge.peepl.be/nl/site/,"Koning Leopold III 115a, 8200 Brugge, België",info@hockeybrugge.be,Recr. Comp.,689.0,51.197226,3.181611
24,Hockey in Hoegaarden,www.hih.be,"Zavelkuilstraat, 3320 Hoegaarden, België",secretaris@hih.be,Recr. Comp.,571.0,50.774203,4.902021


In [21]:
#select top X biggest clubs
df_data_clean_top5=df_data_clean.head(5)
df_data_clean_top5.head(5)

,Club,Website,Adres,E-mail,Hockey,Leden,latitude,longitude
46,R Victory HC,www.victory.be,"Kattenbroek 5, 2650 Edegem, België",info@victory.be,Recr. Comp.,1589.0,51.149110,4.430133
30,KHC Dragons,www.dragons.be,"Gemeentepark 12, 2930 Brasschaat, België",secretaris@dragons.be,Recr. Comp.,1426.0,51.284169,4.488786
0,ARA Gantoise HC,www.gantoise.be,"Tennisstraat 13, 9050 Gent, België",hockey@gantoise.be,Recr. Comp.,1349.0,51.030502,3.766894
3,Braxgata HC,www.braxgata.be,"Schommelei 1-4, 2850 Boom, België",secretaris@braxgata.be,Recr. Comp.,1290.0,51.089335,4.375491
26,Indiana THC,www.thc-indiana.be,"Hondelestraat 13,9840 De Pinte, België",hockey@thc-indiana.be,Recr. Comp.,1103.0,50.993195,3.693569


<h2>Map hockey clubs on the Flanders commune map</h2>

In [22]:
#Plot biggest club as center of the map

address = 'Kattenbroek 5, 2650 Edegem, België'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of HC Victory are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of HC Victory are 51.14911035, 4.4301325531401.


In [23]:
# create map of Flanders with HC Victory in center, using latitude and longitude values
map_victory = folium.Map(location=[latitude, longitude], zoom_start=100)

# add markers to map
for lat, lng, club in zip(df_data_clean['latitude'], df_data_clean['longitude'], df_data_clean['Club']):
    label = '{}'.format(club)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='black',
        fill=True,
        fill_color='#FFFF00',
        fill_opacity=0.7,
        parse_html=False).add_to(map_flanders)  
    
map_flanders

<h1>Intermediate conclusion</h1>

Hockey clubs are spread over Flanders, but there are a few areas where no hockey club is established, and where we would expect potential for starting one.  
If you draw rectangular squares in those areas you can identify center locations in those squares, where distance might prevent youth from practicing hockey.

Let's identify those center locations and then learn more about them.
Building a new hockey club is expensive.  A hockey field costs over 400.000 euro.  Sponsorship of the local commune/city is inevitable.
As the local authorities are inclined to invest only where there is an existing sports infrastructure, let's use foursquare to list the "sport facilities" in the communes where there are no nearby hockey clubs.


<h2>Use Foursquare data to find sports facilities in communes where no hockey clubs are present</h2>

In [25]:
#Let's look at the first possible location.
df_clean.loc[256, 'Gemeente']

'Torhout'

In [34]:
#List cities in areas with no hockey presence
    #visually identified areas with Torhout, Oudenaarde, Scherpenheuvel, Bree as center locations with no hockey-club nearby.
#Let's create a dataframe with these cities and then plot sportsvenues
df_nohockey = df_clean.loc[ [256 , 206, 231, 44 ] , : ]
df_nohockey.head()

,Gemeente,Stad/ Gem.,Provincie,Opper- vlakte (km²),Aantal inwoners (01/01/2016),Deelgemeenten,latitude,longitude
256,Torhout,stad,West-Vlaanderen,4523,20.486,Torhout,51.058591,3.091307
206,Oudenaarde,stad,Oost-Vlaanderen,6806,30.990,"Oudenaarde, Bevere , Edelare , Eine , E...",50.847531,3.607305
231,Scherpenheuvel-Zichem,stad,Vlaams-Brabant,5050,22.833,"Scherpenheuvel , Averbode , Zichem , Mes...",50.979917,4.972987
44,Bree,stad,Limburg,6496,15.785,"Beek , Bree, Gerdingen , Opitter , Tongerlo",51.138624,5.622981


In [37]:
#Starting with Torhout
neighborhood_latitude = df_nohockey.loc[256, 'latitude'] # neighborhood latitude value
neighborhood_longitude = df_nohockey.loc[256, 'longitude'] # neighborhood longitude value

neighborhood_name = df_nohockey.loc[256, 'Gemeente'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Torhout are 51.0585915, 3.09130720454545.


In [27]:
CLIENT_ID = 'ZLSHLAT3WKZAQPYSDTLJLIV12HXGEOPK4LK20HYAYQHBMW23' # your Foursquare ID
CLIENT_SECRET = 'ZAGZ0YF14LUYY3LABPRRM4ID0RVZN4HCD2OE1KWSHMXES0OY' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ZLSHLAT3WKZAQPYSDTLJLIV12HXGEOPK4LK20HYAYQHBMW23
CLIENT_SECRET:ZAGZ0YF14LUYY3LABPRRM4ID0RVZN4HCD2OE1KWSHMXES0OY


In [38]:
# let's get the sports facilities within  5 kilometer of Torhout

LIMIT = 50 # limit of number of venues returned by Foursquare API
radius = 5000 # define radius
search_query='Athletics & Sports'
category="4f4528bc4b90abdf24c9de85"
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&category={}&query={}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    category,
    search_query,
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=ZLSHLAT3WKZAQPYSDTLJLIV12HXGEOPK4LK20HYAYQHBMW23&client_secret=ZAGZ0YF14LUYY3LABPRRM4ID0RVZN4HCD2OE1KWSHMXES0OY&v=20180604&ll=51.0585915,3.09130720454545&category=4f4528bc4b90abdf24c9de85&query=Athletics & Sports&radius=5000&limit=50'

In [39]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5dce626147e0d60022112c82'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'query': 'athletics',
  'totalResults': 70,
  'suggestedBounds': {'ne': {'lat': 51.10359154500004,
    'lng': 3.162769756351632},
   'sw': {'lat': 51.013591454999954, 'lng': 3.019844652739268}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4db2bbba93a017099db32a30',
       'name': 'Zwembad Lichtervelde',
       'location': {'address': 'Zandstraat 15',
        'lat': 51.02755784104803,
        'lng': 3.1348652325928406,
        'labeledLatLngs': [{'label': 'displa

<h2>Write function to get sports facilities for all communes with no hockey presence</h2>

In [48]:
def getNearbyVenues(names, latitudes, longitudes, radius=5000):
    search_query='Athletics & Sports'
    category='4f4528bc4b90abdf24c9de85'
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&category={}&query={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            category,
            search_query,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Gemeente', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
hockey_venues = getNearbyVenues(names=df_nohockey['Gemeente'],
                                   latitudes=df_nohockey['latitude'],
                                   longitudes=df_nohockey['longitude']
                                  )

Torhout
Oudenaarde
Scherpenheuvel-Zichem
Bree


In [73]:
print(hockey_venues.shape)
hockey_venues.head()

(157, 7)


,Gemeente,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Torhout,51.058591,3.091307,Zwembad Lichtervelde,51.027558,3.134865,Athletics & Sports
1,Torhout,51.058591,3.091307,The Move,51.057285,3.098358,Gym / Fitness Center
2,Torhout,51.058591,3.091307,Atletiekpiste,51.070365,3.102275,Athletics & Sports
3,Torhout,51.058591,3.091307,U-FIT,51.076636,3.103840,Gym / Fitness Center
4,Torhout,51.058591,3.091307,Joggingparcours Statieput,51.025567,3.045830,Athletics & Sports


<h2>Communes with most sports infrastructure</h2>

Oudenaarde and Torhout are the communes with most sports infrastructure. 

In [72]:
df_venue_count=hockey_venues.groupby('Gemeente')['Gemeente'].value_counts()
#df.groupby('name')['activity'].value_counts()

df_venue_count.head()

Gemeente               Gemeente             
Bree                   Bree                     24
Oudenaarde             Oudenaarde               50
Scherpenheuvel-Zichem  Scherpenheuvel-Zichem    33
Torhout                Torhout                  50
Name: Gemeente, dtype: int64

In [57]:
#plot sportsfacilities on the Flanders map
# create map of Flanders with Brussels in center, using latitude and longitude values
map_sportsfac = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, club in zip(hockey_venues['Venue Latitude'], hockey_venues['Venue Longitude'], hockey_venues['Venue']):
    label = '{}'.format(club)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='white',
        fill=True,
        fill_color='#FFF000',
        fill_opacity=0.7,
        parse_html=False).add_to(map_flanders)  
    
map_flanders

<h1>Conclusion</h1>

Amongst the 4 communes we identified as being located in an area with no hockey presence, there are 2 which stand out being in an area with the most sports infrastructure.   Oudenaarde and Torhout.    
These communes would be worth additional investigation to determine if a new hockey club would be beneficial.

Foursquare data should be used with caution, especially when selecting on category.   The Athletics and Sport venues also include Sports therapists.  This can be argued and would not benefit this analysis which primarily needed outdoor sports venues.
